# GCSim database loader

Downloads all the teams in the gcsim and exports it as json and a Cypher script for Neo4j DB.

In [47]:
!pip install tqdm
!pip install requests

In [48]:
import requests
import json
from tqdm import tqdm

## Fetch the data from GCSim API

In [49]:

# Fetch all results from the GCSim API
def fetch_all_results():
    base_url = "https://simpact.app/api/db?q="
    limit = 100
    skip = 0
    all_results = []
    
    with tqdm(desc="Fetching data", unit=" items") as pbar:
        while True:
            query = json.dumps({"limit": limit, "skip": skip, "sort": {"create_date": -1}})
            url = f"{base_url}{query}"
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f"Error: {response.status_code}, {response.text}")
                break
            
            data = response.json()
            if not data:  # Stop when no more results
                break
            
            data = data["data"]
            if len(data) == 0:
                break # Stop when no more results

            for item in data:
                all_results.append(item)


            skip += limit  # Move to next page
            pbar.update(len(data))
    
    return all_results

In [50]:
# Fetch all results
all_results = fetch_all_results()
# Save to JSON
with open("gcsim_db.json", "w") as f:
    json.dump(all_results, f)
    f.close()

Fetching data: 3448 items [00:19, 181.26 items/s]


## Load the data from the file

In [51]:
all_results = json.load(open("gcsim_db.json"))
all_characters = {}
all_elements = set()
all_teams = {}

In [52]:
def load_team(team_data):
    summary = team_data["summary"]

    team = {
        "id": team_data["_id"],
        "dps": summary["mean_dps_per_target"],
        "characters": {}
    }

    # Add all characters to the set
    for char in summary["team"]:
        cons = char["cons"] if "cons" in char else 0
        all_characters[char["name"]] = char["element"]
        all_elements.add(char["element"])

        team["characters"][char["name"]] = cons

    all_teams[team["id"]] = team


In [53]:
for team in all_results:
    load_team(team)

## Write the data to Cypher

In [54]:
def write_cypher(fname="gcsim_db.cypher"):
    file = open(fname, "w")
    # Write the elements
    for element in all_elements:
        file.write(f"CREATE (el_{element}:Element {{name: '{element}'}})\n")

    file.write("\n")

    # Write the characters
    for char, element in all_characters.items():
        file.write(f"CREATE (char_{char}:Character {{name: '{char}'}})\n")
        file.write(f"CREATE (char_{char})-[:HAS_ELEMENT]->(el_{element})\n")

    file.write("\n")

    # Write the teams
    for team_id, team in all_teams.items():
        name = []
        for char, cons in team["characters"].items():
            name.append(f"{char} (C{cons})")
        
        name = ", ".join(name)

        file.write(f"CREATE (team_{team_id}:Team {{id: \"{team_id}\", name: \"{name}\", dps: {team['dps']}}})\n")

        for char, cons in team["characters"].items():
            file.write(f"CREATE (char_{char})-[:IN_TEAM {{cons: {cons}}}]->(team_{team_id})\n")

    file.write(";\n")
    file.close()

In [55]:
write_cypher()